## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [2]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [5]:
## your code here
debar = pd.read_csv("~/Desktop/QSS20_S25/problemsets/pset2/pset2_inputdata/debar.csv")
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [7]:
## your code here
len(debar)
unique_employers = debar["Name"].nunique()
unique_employers

114

98

Since there are 114 rows and 98 employers there must be multiple rows for some employers

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [10]:
## your code here
#A
repeated_names = debar["Name"].value_counts()
repeated_names = repeated_names[repeated_names > 1].index

debar["is_repeated"] = debar["Name"].isin(repeated_names)

In [11]:
#B
print(debar[debar["is_repeated"]==True])

                                        Name       City, State  \
6                    Annabella Land & Cattle     Annabella, UT   
7                        Autumn Hill Orchard        Groton, MA   
8      Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
11                     Loewen Harvesting LLC   Brownsville, TX   
12               Rollo Farm Labor Contractor         Miami, FL   
14                             Sharon Mathis        Tifton, GA   
15                                 SRT Farms        Morton, TX   
16                               Mark Duncan     Roosevelt, UT   
17          Maple Ridge Custom Services, LLC     Altheimer, AK   
18                                 F&W Farms       Ingalls, KS   
19                        Cisco Produce Inc.         Cairo, GA   
21   Old Tree Farms/Verpaalen Custom Service         Volga, SD   
24               Rollo Farm Labor Contractor         Miami, FL   
25                     Loewen Harvesting LLC    Brownfield, TX   
28     Cad

In [12]:
#C
mult_debar= debar[debar["is_repeated"]==True]
mult_debar.head()
mult_debar.shape

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [14]:
## your code here
mult_debar_sorted = mult_debar.sort_values(["Name", "City, State", "Start date"])

mult_debar_sorted["violation_number"] = mult_debar_sorted.groupby(["Name", "City, State"]).cumcount() + 1

mult_debar_wide = mult_debar_sorted.pivot(
    index=["Name", "City, State"],
    columns="violation_number",
    values="Start date"
)

mult_debar_wide.columns = ["start_date_viol1","start_date_viol2"]

mult_debar_wide.head()
mult_debar_wide.shape

start_date_viol1  \
Name                                  City, State                        
Annabella Land & Cattle               Annabella, UT           5/9/2014   
                                      Annabella, Utah         5/9/2014   
Autumn Hill Orchard                   Groton, MA              7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX              7/20/2014   
                                      Caddo, Texas           7/20/2014   

                                                      start_date_viol2  
Name                                  City, State                       
Annabella Land & Cattle               Annabella, UT                NaN  
                                      Annabella, Utah              NaN  
Autumn Hill Orchard                   Groton, MA              7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX                    NaN  
                                      Caddo, Texas                 NaN

(20, 2)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [16]:
## your code here
#A
mult_debar_wide["is_dup"] = mult_debar_wide["start_date_viol1"] == mult_debar_wide["start_date_viol2"]
mult_debar_wide = mult_debar_wide.reset_index()

In [17]:
debar = debar.sort_values(["Name", "City, State", "Start date"])
debar["violation_number"] = debar.groupby(["Name", "City, State"]).cumcount() + 1

#merge dup info from mult_debar_wide into full debar
debar = pd.merge(
    debar,
    mult_debar_wide[["Name", "City, State", "is_dup"]],
    on=["Name", "City, State"],
    how="left"
)

debar["is_dup"] = debar["is_dup"].fillna(False)

#keep only viol 1 for dups, all for others
debar_clean = pd.concat([
    debar[(debar["is_dup"] == True) & (debar["violation_number"] == 1)],
    debar[debar["is_dup"] == False]
], axis=0)

debar_clean.shape
debar_clean["Name"].nunique()


/var/folders/9_/fcykl83s1dncqz69y6vx8b6w0000gn/T/ipykernel_23150/91469115.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  debar["is_dup"] = debar["is_dup"].fillna(False)


(105, 9)

98

# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset2_inputdata`

In [20]:
# your code here to load the data 

jobs_clean = pd.read_csv("~/Desktop/QSS20_S25/problemsets/pset2/pset2_inputdata/jobs.csv")
jobs_clean.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [22]:
## your code here
inner = jobs_clean.merge(debar_clean, left_on="EMPLOYER_NAME", right_on="Name", how="inner", indicator=True)

In [23]:
print(inner)

          CASE_NUMBER                           CASE_STATUS  \
0  H-300-20287-876656  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    Y                 Seasonal   

  EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  ...            Name  \
0                Y  Rafael Barajas            NaN  ...  Rafael Barajas   

        City, State                         Violation Duration Start date  \
0  Sebring, Florida  Non-payment of certification fee   1 year  9/23/2016   

    End date is_repeated  violation_number  is_dup  _merge  
0  9/22/2017       False               1.0   False    both  

[1 rows x 148 columns]


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [26]:
## insert your code to turn into uppercase here
employer_name_2 = [ name.upper() for name in jobs_clean["EMPLOYER_NAME"]]
name_2 = [ name.upper() for name in debar_clean["Name"]]
#employer_name_2, name_2

In [27]:
## insert your code for the random sample
print(random.sample(employer_name_2, 15))
print(random.sample(name_2, 15))

['BLUE RIDGE MOUNTAIN DAIRY LLC  ', 'MARK & CHARLYNN LIUZZA FARMS, LLC', 'DANIEL MARCANTEL FARMS, INC', 'BROWN BROTHERS FARMS', 'AMERIGO FARMS, INC.  ', 'JASON LEE YOUNG', 'LINDSKOV-THIEL CHAROLAIS RANCH', 'PREMIUM EMPLOYMENT SERVICES, INC. ', 'BEKKER FARMS', 'STRIPPIN FEATHERS, LLC', 'DORE FARMS', '6 M FARMS, LLC/JAMILLE R. MENARD', 'DEBERARD CATTLE COMPANY LLC.', "FRUGE' AQUAFARMS, INC.", 'LIUZZA PRODUCE FARM, INC.']
['AB RANCH', 'TURNER FARMS', 'AGECY I LLC', 'CIRA CORTEZ LOPEZ', 'J&J HARVESTING', 'LESLIE RENEE DREW', 'TRAVIS AND TARA LAMBOURN', 'YESENIA PEREZ', 'DONNA LUCIO', 'JIM AND ANN SHIPLEY WILLIAM SHIPLEY*', 'CADDO CREEK RANCH, DBA PARADISE RANCH', 'WALKER PLACE', 'PETER PETERS', 'JOHN & NETA LEOPKY FARMS', 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE']


In [28]:
## insert your code for assigning the uppercase names back to the data
jobs_clean["EMPLOYER_NAME"] = employer_name_2
debar_clean["Name"] = name_2

jobs_clean.head()
debar_clean.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"FAZIO FARMS OPERATING COMPANY, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,CHARLIE SUNDERLAND,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,MICHAEL RUDEBUSCH,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,LODAHL FARMS,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"DUNSON HARVESTING, INC.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,violation_number,is_dup
10,AUTUMN HILL ORCHARD,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True,1.0,True
30,DOVE CREEK FARMS,"Mount Vernon, TX",Failure to respond to audit request,2 years,2/9/2018,2/8/2018,True,1.0,True
34,F&W FARMS,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True,1.0,True
64,MACKY AND BRAD FARMS,"Plains, TX",Failure to respond to audit (no response),1 year,2/13/2015,2/12/2016,True,1.0,True
68,MARK DUNCAN,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,1.0,True


### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [30]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [31]:
## insert your code here with the regex pattern for part A
regex = r"(?<=(LLC|INC| CO))\."

## insert your code to use re.sub to apply the pattern to the test cases for part B
test_1 = re.sub(regex, "", pos_example_1)
test_2 = re.sub(regex, "", pos_example_2)
test_3 = re.sub(regex, "", neg_example)
test_1, test_2, test_3

('CISCO PRODUCE INC', 'AVOYELLES HONEY CO, LLC', 'E.V. RANCH LLP')

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [63]:
## your code here to clean the columns
employer_name_3 = [ re.sub(regex, "", name) for name in jobs_clean["EMPLOYER_NAME"]]
name_3 = [ re.sub(regex, "", name)  for name in debar_clean["Name"]]
name_3, employer_name_3

(['AUTUMN HILL ORCHARD',
  'DOVE CREEK FARMS',
  'F&W FARMS',
  'MACKY AND BRAD FARMS',
  'MARK DUNCAN',
  'ROLLO FARM LABOR CONTRACTOR',
  'SRT FARMS',
  'SHARON MATHIS',
  'TURNER FARMS',
  '69 FARMS LLC',
  'AB RANCH',
  'ABE- NANCY FROESE',
  'AGECY I LLC',
  'AGECY II, LLC',
  'ALTENDORF TRANSPORT INC',
  'ALTERIC JEAN-CHARLES',
  'ANNABELLA LAND & CATTLE',
  'ANNABELLA LAND & CATTLE',
  'ANTON FERTILIZER INC',
  'AVOYELLES HONEY CO, LLC',
  'B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)',
  'C HANGING L RANCH LLP',
  'CADDO CREEK RANCH, DBA PARADISE RANCH',
  'CADDO CREEK RANCH, DBA PARADISE RANCH',
  'CHRIS ROBINSON',
  'CIRA CORTEZ LOPEZ',
  'CISCO PRODUCE INC',
  'CISCO PRODUCE INC',
  'CITY PINESTRAW AND HARVESTING',
  'COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*',
  'DANECO, LLC',
  'DAVID C. MARTINEZ',
  'DAVID JACKSON',
  'DEAL FAMILY FARM',
  'DELIA ROJAS',
  'DIXIE BELL #2',
  'DONNA LUCIO',
  'E.V. RANCH LLP',
  'EVERGREEN PRODUCE',
  'FIRST AMERICAN HO

In [34]:
jobs_clean["name_clean"] = employer_name_3
debar_clean["name_clean"] = name_3

In [35]:
jobs_clean["name_changed"] = jobs_clean["EMPLOYER_NAME"].str.upper() != jobs_clean["name_clean"]
debar_clean["name_changed"] = debar_clean["Name"].str.upper() != debar_clean["name_clean"]

In [36]:
## your code here to print the head
job_name_changed = jobs_clean[jobs_clean["name_changed"]]
debar_name_changed = debar_clean[debar_clean["name_changed"]]

print(debar_name_changed[["Name", "name_clean"]])
print(job_name_changed[["EMPLOYER_NAME", "name_clean"]])

                                Name                       name_clean
5           ALTENDORF TRANSPORT INC.          ALTENDORF TRANSPORT INC
9              ANTON FERTILIZER INC.             ANTON FERTILIZER INC
12          AVOYELLES HONEY CO., LLC          AVOYELLES HONEY CO, LLC
19                CISCO PRODUCE INC.                CISCO PRODUCE INC
20                CISCO PRODUCE INC.                CISCO PRODUCE INC
39                GERONIMO SHEEP CO.                GERONIMO SHEEP CO
42  GREAT PLAINS FLUID SERVICE, INC.  GREAT PLAINS FLUID SERVICE, INC
80                       PROMAX INC.                       PROMAX INC
85                     REIMER'S INC.                     REIMER'S INC
86                     REIMER’S INC.                     REIMER’S INC
94      SAXTONS RIVER ORCHARDS, INC.      SAXTONS RIVER ORCHARDS, INC
                                 EMPLOYER_NAME  \
4                      DUNSON HARVESTING, INC.   
7     FARM LABOR ASSOCIATION FOR GROWERS, INC.   
14        

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [38]:
## your code here
inner_2 = jobs_clean.merge(debar_clean, left_on="name_clean", right_on="name_clean", how="inner", indicator=True)


In [86]:
inner_2

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,"City, State",Violation,Duration,Start date,End date,is_repeated,violation_number,is_dup,name_changed_y,_merge
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False,1.0,False,False,both


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here